In [28]:
import pandas as pd
from PIL import Image
from skimage import color

import numpy as np
# from colour import XYZ_to_CAM16
import numpy as np
# from colour.models import RGB_to_XYZ
# 读取图片
img = Image.open('image/101393174_p0.png')

# 获取图片尺寸
width, height = img.size

# 获取像素数据
pixels = list(img.getdata())

# 创建DataFrame,A代表透明度
try:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B','A'])
    df = df.drop(columns=['A'])
except:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B'])
# 添加X和Y列
df['X'] = df.index % width
df['Y'] = df.index // width

# 显示DataFrame
print(df)

           R    G    B     X     Y
0         97  218  228     0     0
1         97  218  228     1     0
2         97  218  228     2     0
3         96  218  227     3     0
4         96  218  228     4     0
...      ...  ...  ...   ...   ...
5912995   97  143  120  1966  2999
5912996  103  140  121  1967  2999
5912997   98  149  119  1968  2999
5912998   96  147  119  1969  2999
5912999  105  144  124  1970  2999

[5913000 rows x 5 columns]


In [29]:
from colorspacious import cspace_converter
RGB = np.array([df['R'], df['G'], df['B']]).T

converter = cspace_converter("sRGB255", "CIELCh")
# 将LCH颜色添加到DataFrame
lch = converter(RGB)
df['L'] = lch[:, 0]
df['C'] = lch[:, 1]
df['H'] = lch[:, 2]
df['L'] = df['L'].round()
df['C'] = df['C'].round()
df['H'] = df['H'].round()
df['count'] = 1
df


,R,G,B,X,Y,L,C,H,count
0,97,218,228,0,0,81.0,34.0,206.0,1
1,97,218,228,1,0,81.0,34.0,206.0,1
2,97,218,228,2,0,81.0,34.0,206.0,1
3,96,218,227,3,0,81.0,35.0,205.0,1
4,96,218,228,4,0,81.0,35.0,206.0,1
...,...,...,...,...,...,...,...,...,...
5912995,97,143,120,1966,2999,56.0,22.0,161.0,1
5912996,103,140,121,1967,2999,55.0,18.0,161.0,1
5912997,98,149,119,1968,2999,57.0,26.0,156.0,1
5912998,96,147,119,1969,2999,57.0,25.0,158.0,1


In [30]:
C_mean = df['C'].mean()
L_mean = df['L'].mean()
H_mean = df['H'].mean()

In [31]:
df_mean = df.groupby('H').agg({'R': 'mean', 'G': 'mean', 'B': 'mean','L': 'mean','C':'mean'}).reset_index()
df_mean['R'] = df_mean['R'].round().astype(int)
df_mean['G'] = df_mean['G'].round().astype(int)
df_mean['B'] = df_mean['B'].round().astype(int)
df_mean['L'] = df_mean['L'].round().astype(int)
df_mean['C'] = df_mean['C'].round().astype(int)
df_count = df.groupby([ 'H']).size().reset_index(name='v')
df_merged = pd.merge(df_mean, df_count, on='H')
df_merged

,H,R,G,B,L,C,v
0,0.0,62,47,51,21,7,654
1,1.0,127,96,105,44,14,568
2,2.0,131,101,109,45,14,633
3,3.0,139,109,116,48,13,654
4,4.0,129,98,105,44,14,672
...,...,...,...,...,...,...,...
356,356.0,139,108,119,48,14,598
357,357.0,129,101,111,45,13,655
358,358.0,131,101,110,45,14,607
359,359.0,140,113,121,50,12,640


In [32]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按色相顺序）', showlegend=False)
fig.show()

In [33]:
fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none'))
fig.update_layout(title='色相分布图（按每个色相份额排序）', showlegend=False)
fig.show()

In [34]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('L')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按亮度排序）', showlegend=False)
fig.show()

In [35]:




import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

# Calculate LCH values based on C_mean, L_mean, and H
lch_colors = [(L_mean, C_mean, h) for h in df_merged_sorted['H']]

# Convert LCH colors to RGB
converter = cspace_converter("CIELCh", "sRGB255")
rgb_colors = converter(lch_colors)

# Format RGB colors
colors = ['rgb({},{},{})'.format(int(r), int(g), int(b)) for r, g, b in rgb_colors]

fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图(平均色度和亮度后)', showlegend=False)
fig.show()

In [36]:
df_learn = df.groupby(['C', 'H']).agg({ 'L': 'sum'}).reset_index()
df_learn.columns = ['C', 'H', 'L']
df_learn['C_36']= df_learn['C']*3.6

In [37]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=256, init='k-means++',algorithm='lloyd')
kmeans.fit(df_learn[['C_36', 'H',]], sample_weight=df_learn['L'])
# 获取聚类结果
df_predict = df.copy()
df_predict['C_36'] = df_predict['C']*3.6
labels = kmeans.predict(df_predict[['C_36', 'H']])
# 将聚类结果写回原始数据框中

In [38]:
df['Cluster'] = labels
df

,R,G,B,X,Y,L,C,H,count,Cluster
0,97,218,228,0,0,81.0,34.0,206.0,1,30
1,97,218,228,1,0,81.0,34.0,206.0,1,30
2,97,218,228,2,0,81.0,34.0,206.0,1,30
3,96,218,227,3,0,81.0,35.0,205.0,1,226
4,96,218,228,4,0,81.0,35.0,206.0,1,30
...,...,...,...,...,...,...,...,...,...,...
5912995,97,143,120,1966,2999,56.0,22.0,161.0,1,77
5912996,103,140,121,1967,2999,55.0,18.0,161.0,1,5
5912997,98,149,119,1968,2999,57.0,26.0,156.0,1,58
5912998,96,147,119,1969,2999,57.0,25.0,158.0,1,58


In [39]:
df_learn_show = df.groupby(['Cluster']).agg({ 'L': 'sum','C': 'mean','H': 'mean','R': 'mean', 'G': 'mean', 'B': 'mean','count': 'sum'}).reset_index()
df_learn_show['C'] = df_learn_show['C'].round().astype(int)
df_learn_show['H'] = df_learn_show['H'].round().astype(int)
df_learn_show['R'] = df_learn_show['R'].round().astype(int)
df_learn_show['G'] = df_learn_show['G'].round().astype(int)
df_learn_show['B'] = df_learn_show['B'].round().astype(int)

df_learn_show

,Cluster,L,C,H,R,G,B,count
0,0,3845889.0,32,148,143,198,153,51254
1,1,1344347.0,13,228,54,88,99,38982
2,2,2474088.0,29,203,103,199,203,33161
3,3,2571673.0,11,258,198,216,234,30202
4,4,2145369.0,42,151,89,169,115,34143
...,...,...,...,...,...,...,...,...
251,251,359223.0,38,271,58,121,182,7256
252,252,69380.0,82,138,105,241,104,809
253,253,3511948.0,32,138,182,225,172,41133
254,254,3174645.0,27,153,124,177,142,47061


In [40]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色彩进行聚类后的结果',
    xaxis=dict(title='C'),
    yaxis=dict(title='H'),
    showlegend=False
)

fig.show()


In [41]:
from sklearn.cluster import KMeans
kmeans2 = KMeans(n_clusters=5, init='k-means++',algorithm='lloyd')
kmeans2.fit(df_learn_show[['H','C']],sample_weight=df_learn_show['L'])
# 获取聚类结果
labels2 = kmeans2.predict(df_learn_show[[ 'H','C']])
df_learn_show['Cluster2'] = labels2
df_learn_show
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

,Cluster,L,C,H,R,G,B,count,Cluster2
0,0,3845889.0,32,148,143,198,153,51254,1
1,1,1344347.0,13,228,54,88,99,38982,4
2,2,2474088.0,29,203,103,199,203,33161,4
3,3,2571673.0,11,258,198,216,234,30202,2
4,4,2145369.0,42,151,89,169,115,34143,1
...,...,...,...,...,...,...,...,...,...
251,251,359223.0,38,271,58,121,182,7256,2
252,252,69380.0,82,138,105,241,104,809,1
253,253,3511948.0,32,138,182,225,172,41133,1
254,254,3174645.0,27,153,124,177,142,47061,1


In [42]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show['Cluster2'],
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色相聚一次类',
    xaxis=dict(title='C：色度'),
    yaxis=dict(title='H：色相'),
    showlegend=False
)

fig.show()

In [43]:
df_learn_show['final_v']=df_learn_show['C']*df_learn_show['L']

In [44]:
df_learn_l = df.groupby(['L']).agg({ 'count': 'sum'}).reset_index()

In [45]:
df_learn_show['L'] = (df_learn_show['L']/df_learn_show['count']).round().astype(int)

In [46]:
from sklearn.cluster import KMeans
kmeans3 = KMeans(n_clusters=10, init='k-means++',algorithm='lloyd')
kmeans3.fit(df_learn_l[['L']],sample_weight=df_learn_l['count'])
# 获取聚类结果
labels3 = kmeans3.predict(df_learn_l[[ 'L']])
df_learn_l['Cluster3'] = labels3
df_learn_l
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

,L,count,Cluster3
0,0.0,459,6
1,1.0,268,6
2,2.0,313,6
3,3.0,432,6
4,4.0,606,6
...,...,...,...
96,96.0,59316,7
97,97.0,43177,7
98,98.0,36210,7
99,99.0,30189,7


In [47]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_l['L'],
    y=df_learn_l['count'],
    mode='markers',
    marker=dict(
        color=df_learn_l['Cluster3'],
        colorscale='Viridis',
        opacity=0.8
    )
))

fig.update_layout(
    title='在亮度上对像素聚类',
    xaxis=dict(title='L：亮度'),
    yaxis=dict(title='像素量'),
    showlegend=False
)

fig.show()


In [48]:
df_learn_l

,L,count,Cluster3
0,0.0,459,6
1,1.0,268,6
2,2.0,313,6
3,3.0,432,6
4,4.0,606,6
...,...,...,...
96,96.0,59316,7
97,97.0,43177,7
98,98.0,36210,7
99,99.0,30189,7


In [49]:
df_l_plan = df_learn_l.groupby('Cluster3').apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)
df_l_plan = df_l_plan.apply(lambda x: x.sort_values().reset_index(drop=True))
df_l_plan

/tmp/ipykernel_157548/210686816.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,L,count,Cluster3
0,18.0,25084.0,0.0
1,30.0,61685.0,1.0
2,39.0,71688.0,2.0
3,47.0,78078.0,3.0
4,48.0,83913.0,4.0
5,60.0,85339.0,5.0
6,66.0,85733.0,6.0
7,73.0,92527.0,7.0
8,90.0,97227.0,8.0
9,100.0,97769.0,9.0


In [50]:
df_color_plan = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)
df_color_plan

/tmp/ipykernel_157548/2810096399.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,11.0,51.0,28.0,168.0,67.0,134.0,112.0,65197.0,0.0,93340156.0
1,193.0,64.0,36.0,154.0,101.0,171.0,127.0,66298.0,1.0,153633420.0
2,12.0,55.0,18.0,268.0,112.0,134.0,164.0,46354.0,2.0,45679194.0
3,222.0,82.0,38.0,89.0,231.0,203.0,134.0,2863.0,3.0,8971952.0
4,30.0,77.0,33.0,207.0,91.0,205.0,215.0,32060.0,4.0,81117597.0


In [51]:
df_plan = pd.DataFrame(columns=['L','C','H','R','G','B','count','count2','cluster2','cluster3'])

converter2 = cspace_converter("CIELCh","sRGB255")
# 将LCH颜色添加到DataFrame

lch = converter(RGB)
for idx, data in df_color_plan.iterrows():
    for idx2, data2 in df_l_plan.iterrows():
        L = data2['L']
        C = data['C']
        H = data['H']
        RGB = converter2((L, C, H))
        R = RGB[0]
        G = RGB[1]
        B = RGB[2]
        count = data['count']
        count2 = data2['count']
        cluster2 = data['Cluster2']
        cluster3 = data2['Cluster3']
        df_plan.loc[len(df_plan)] = [L,C,H,R,G,B,count,count2,cluster2,cluster3]

df_plan = df_plan.applymap(lambda x: 0 if x < 0 else x)
df_plan


/tmp/ipykernel_157548/1174046150.py:22: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,L,C,H,R,G,B,count,count2,cluster2,cluster3
0,18.0,28.0,168.0,0.000000,52.997773,34.936278,65197.0,25084.0,0.0,0.0
1,30.0,28.0,168.0,0.576471,81.080154,60.881736,65197.0,61685.0,0.0,1.0
2,39.0,28.0,168.0,33.322295,103.133631,81.630428,65197.0,71688.0,0.0,2.0
3,47.0,28.0,168.0,55.371126,123.380118,100.864539,65197.0,78078.0,0.0,3.0
4,48.0,28.0,168.0,58.025031,125.951030,103.316866,65197.0,83913.0,0.0,4.0
5,60.0,28.0,168.0,89.508695,157.453842,133.509142,65197.0,85339.0,0.0,5.0
6,66.0,28.0,168.0,105.305528,173.632708,149.098602,65197.0,85733.0,0.0,6.0
7,73.0,28.0,168.0,123.933352,192.844705,167.668064,65197.0,92527.0,0.0,7.0
8,90.0,28.0,168.0,170.301341,240.905719,214.332032,65197.0,97227.0,0.0,8.0
9,100.0,28.0,168.0,198.375290,270.037412,242.728654,65197.0,97769.0,0.0,9.0


In [52]:
df_color_plan

,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,11.0,51.0,28.0,168.0,67.0,134.0,112.0,65197.0,0.0,93340156.0
1,193.0,64.0,36.0,154.0,101.0,171.0,127.0,66298.0,1.0,153633420.0
2,12.0,55.0,18.0,268.0,112.0,134.0,164.0,46354.0,2.0,45679194.0
3,222.0,82.0,38.0,89.0,231.0,203.0,134.0,2863.0,3.0,8971952.0
4,30.0,77.0,33.0,207.0,91.0,205.0,215.0,32060.0,4.0,81117597.0


In [53]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_plan['cluster3'],
    y=df_plan['cluster2'],
    mode='markers',
    marker=dict(
        size=32,
        color=df_plan.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_plan.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='配色方案',
    xaxis=dict(title='亮度'),
    yaxis=dict(title='颜色'),
    showlegend=False
)

fig.show()

In [54]:
import plotly.graph_objects as go

# Group the DataFrame by 'Cluster2' and select the row with the maximum 'L' value
df_max_l = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)

# Create the pie chart
fig = go.Figure(data=go.Pie(
    labels=df_max_l['Cluster2'],
    values=df_max_l['count'],
    marker=dict(
        colors=['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_max_l['R'], df_max_l['G'], df_max_l['B'])]
    ),
    textinfo='none'
))

fig.update_layout(title='取色结果（如果图片颜色太少区别会很小，这不是bug是feature😡）', showlegend=False)
fig.show()


/tmp/ipykernel_157548/1545999088.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

